In [6]:
import pandas as pd
import numpy as np

Create control file first.
This controls most parameters to be used in the simulation. 
Default parameters are assigned a value. 
Functionality can be added to allow advanced users to change default values.

In [2]:
#numeric parameters inputs
keyword = input("What kind of scenario do you want to simulate?")
ubulk = float(input("What is the ubulk value?"))
mesh_size = float(input("What is the mesh size?"))
fluid = input("What will be the main fluid, water or air?")
differencing = input("What discretisation scheme do you choose, 2nd order CDS, 4th order CDS, or WENO?")
dt = float(input("Which timestep do you want?"))
itime_end = int(input("How many timesteps do you want to run?"))
restart = bool(input("Do you want a restart?"))
reinitmean = input("Do you want to calculate statistics from scratch?")
n_out = int(input("How often do you want to produce outputs (tecbins)?"))
l_transient = input("Do you want to produce transient outputs?")
if (l_transient == "yes"):
    results = int(input("How often do you want to produce transient outputs?"))
#numeric parameters defaults
pr = 0.72
turb_schmidt = 0.6
beta = 0.0034
gx = 0
gy = 0
gz = -9.81
convection_scheme = 3
diffusion_scheme = 3
solver = 2
multigrid_step = 1
multigrid_iteration_scheme = 2
multigrid_max_iter = 30
restriction_iter = 2
prolongation_iter = 1
variable_fixed = "F"
sweeps = 25
safety_factor = 0.2
niter = 20
eps = 1.0E-5
nswp_1 = 5
nswp_2 = 5
nswp_3 = 5
nswp_4 = 20

What kind of scenario do you want to simulate? dgk
What is the ubulk value? skdgh


ValueError: could not convert string to float: 'skdgh'

In [4]:
#flow boundary inputs
west_boundary_condition = input("""Which west boundary condition option do you want: inflow,slip, no-slip, 
                                law of the wall (smooth log law),1/7th power law, prescribed inflow, SEM, 
                                periodic condition, rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
east_boundary_condition = input("""Which east boundary condition option do you want: outflow, convective, periodic, neumann,
                                slip, no-slip,smooth log law, rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
south_boundary_condition = input("""Which south boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
north_boundary_condition = input("""Which north boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
bottom_boundary_condition = input("""Which bottom boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
top_boundary_condition = input("""Which top boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law?""")
#flow boundary defaults
friction_coeff = 'n'
only_if_bc = 62
save_inflow_data = "F"
inlet_num = 5000

Which west boundary condition option do you want: inflow,slip, no-slip, 
                                law of the wall (smooth log law),1/7th power law, prescribed inflow, SEM, 
                                periodic condition, rough log law, 1/6th law, 1/7th law, or 1/8th law? 02
Which east boundary condition option do you want: outflow, convective, periodic, neumann,
                                slip, no-slip,smooth log law, rough log law, 1/6th law, 1/7th law, or 1/8th law? askdgih
Which south boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law? saldghaw
Which north boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough log law, 1/6th law, 1/7th law, or 1/8th law? sadghwe
Which bottom boundary condition option do you want: slip, no-slip, periodic, smooth log law, 
                                rough lo

In [ ]:
#synthetic eddy method defaults
velocity_profile = 12
turbulence_intensity = 0.1
inlet_profile_num = 1000

In [ ]:
#modelling options inputs
t_start_averaging_1 = float(input("At which ctime do you want to start collecting statistics?"))
SGS_model = input("Do you want an SGS model?")
if (SGS_model == "yes"):
    SGS_model_choice = input("Which SGS model do you want: Smagorinsky, WALE, OneEqnModel, or eps model (RANS)?")
limb = input("Do you want immersed boundaries to be on or off?")
lenergy = input("Do you want energy equation?")
lpt = input("Do you want to create lagrangian particles?")
lscalar = input("Passive scalar? y/n")
l_active_scalar = input("Active scalar? y/n")
#modelling options defaults
time_averaging = "T"
t_start_averaging_2 = t_start_averaging_1 + 1
noise = 0.0
LMR = 2
normal_ghost_velocity_interpolation = 2
lrough = "F"
openmp_threads = 1
pl_ex = 2
th = 293
tc = 293

In [ ]:
#energy boundary conditions defaults
west_temp_boundary = 7
east_temp_boundary = 7
south_temp_boundary = 7
north_temp_boundary = 7
bottom_temp_boundary = 8
top_temp_boundary = 9

In [ ]:
#free surface parameters defaults
l_lsm = "F"
reinit = "F"
ntime_reinit = 3
reldif_LSm = 0.001
length = 0.02
accuracy = 22
#TODO: ask about this value
missing_value = 0.65
lends = "F"
L_LSMbase = "F"
L_LSMinit = "F"
L_anim_phi = "F"
L_anim_grid = "F"
densl = 1000.0
densg = 1.205
nul = 1.0E-6
nug = 1.5E-5
gx = 0.0
gy = 0.0 
gz = -9.8
slope = 0.00

In [ ]:
#time series defaults
num_time_series_points = 1
point_num_1 = 1
point_num_1_2 = 10
point_num_1_3 = 10
point_num_1_4 = 10

Infodom file gives coordinates for sub-domains. A few options are default.
Parameters are calculated based on user input. 
Need to update this because not all values of grid size will result in whole number for cells 

In [9]:
print("Let's create the infodom file.")
grid_size_x = float(input("What is the grid size for x?"))
grid_size_y = float(input("What is the grid size for y?"))
grid_size_z = float(input("What is the grid size for z?"))

domain_length = float(input("How long is your domain in meters?"))
domain_width = float(input("How wide is your domain in meters?"))
domain_height = float(input("How high is your domain in meters?"))

#user input for sub-domain, TO DO: ask about this, if it's user input or pre-determined
sub_domain_i = int(input("How many sub-domains do you want in the x direction?"))
sub_domain_j = int(input("How many sub-domains do you want in the y direction?"))
sub_domain_k = int(input("How many sub-domains do you want in the z direction?"))

print("Calculating...")
#calculation for number of cells for whole domain
#TODO try catch each one must be whole number, iteratively adjust domain_length, grid size until converge on whole numbers for each
cell_num_x = domain_length/grid_size_x
cell_num_y = domain_width/grid_size_y
cell_num_z = domain_height/grid_size_z

#calculation for cells per block
#TODO try catch less than 100 cells per block
#TODO try catch each one must be whole number, iteratively adjust sub_domains, grid size until converge on whole numbers for each
cells_per_block_x = cell_num_x/sub_domain_i
cells_per_block_y = cell_num_y/sub_domain_j
cells_per_block_z = cell_num_z/sub_domain_k


#create df with 6 columns and 4 rows from infodom.cin example
infodom_initial_df = pd.DataFrame({'Grid Size': [grid_size_x,grid_size_y,grid_size_z,np.nan],
                                   'Overall Domain Size': [domain_length,domain_width,domain_height,np.nan],
                                   'Number of Cells for Whole Domain': [cell_num_x,cell_num_y,cell_num_z,cell_num_x*cell_num_y*cell_num_z],
                                   'Sub-domains': [sub_domain_i,sub_domain_j,sub_domain_k,sub_domain_i*sub_domain_j*sub_domain_k],
                                   'Cells per Block': [cells_per_block_x,cells_per_block_y,cells_per_block_z,cells_per_block_x*cells_per_block_y*cells_per_block_z]})

Let's create the infodom file.


What is the grid size for x? 0.01
What is the grid size for y? 0.01
What is the grid size for z? 0.01
How long is your domain in meters? 4
How wide is your domain in meters? 1
How high is your domain in meters? 0.5
How many sub-domains do you want in the x direction? 5
How many sub-domains do you want in the y direction? 2
How many sub-domains do you want in the z direction? 2


Calculating...


In [10]:
#generating input for the infodom.cin file
#specify coordinates of each block
#individual block dimension calculation
block_dimension_x = domain_length/sub_domain_i
block_dimension_y = domain_width/sub_domain_j
block_dimension_z = domain_height/sub_domain_k

#get total block num
total_block_num = sub_domain_i*sub_domain_j*sub_domain_k

#new df
#column 1 is a list iterating total_block_num times
domain = list(range(total_block_num))
rdiv = [1]*total_block_num
print(block_dimension_x)

0.8


In [11]:
def get_coords(sub_domain,block_dimension):
    #x1 and x2 for now, y1 and y2, z1 and z2
    #loop sub_domain times
    #counter x1 from 0
    counter = 0
    v1 = []
    v2 = []
    for i in range(sub_domain):
        v1.append(round(counter,2))
        counter = counter+block_dimension
        v2.append(round(counter,2))

    return v1,v2

In [15]:
x1,x2 = get_coords(sub_domain_i,block_dimension_x)
y1,y2 = get_coords(sub_domain_j,block_dimension_y)
z1,z2 = get_coords(sub_domain_k,block_dimension_z)

print(x1)
print(sub_domain_i)
print(x1*sub_domain_j*sub_domain_k)
print(x1*(len(y1)*len(z1)))

[0, 0.8, 1.6, 2.4, 3.2]
5
[0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2]
[0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2, 0, 0.8, 1.6, 2.4, 3.2]


In [20]:
# Repeat coordinates for all blocks so they are sitting in different spots in the 3D space
x1_repeated = x1 * (len(y1) * len(z1))
x2_repeated = x2 * (len(y1) * len(z1))

y1_repeated = [y for _ in range(len(z1)) for y in y1 for _ in range(len(x1))]
y2_repeated = [y for _ in range(len(z1)) for y in y2 for _ in range(len(x1))]

z1_repeated = [z for z in z1 for _ in range(len(y1) * len(x1))]
z2_repeated = [z for z in z2 for _ in range(len(y1) * len(x1))]

In [21]:
infodom_coord_df = pd.DataFrame({'Domain':domain,'rdiv':rdiv,'x1':x1_repeated,'x2':x2_repeated,
                                'y1':y1_repeated,'y2':y2_repeated,'z1':z1_repeated,'z2':z2_repeated})
print("Finished coordinate calculation.")
infodom_coord_df.head(20)

,Domain,rdiv,x1,x2,y1,y2,z1,z2
0,0,1,0.0,0.8,0.0,0.5,0.00,0.25
1,1,1,0.8,1.6,0.0,0.5,0.00,0.25
2,2,1,1.6,2.4,0.0,0.5,0.00,0.25
3,3,1,2.4,3.2,0.0,0.5,0.00,0.25
4,4,1,3.2,4.0,0.0,0.5,0.00,0.25
5,5,1,0.0,0.8,0.5,1.0,0.00,0.25
6,6,1,0.8,1.6,0.5,1.0,0.00,0.25
7,7,1,1.6,2.4,0.5,1.0,0.00,0.25
8,8,1,2.4,3.2,0.5,1.0,0.00,0.25
9,9,1,3.2,4.0,0.5,1.0,0.00,0.25


In [166]:
def create_infodom():
    
    #Update user on status of application
    print('Writing file...')
    # Hard-code the length of the equal signs line to 49
    equals_line = "=" * 49

    header = f"{total_block_num} number of domains\n"
    footer = f"{sub_domain_i} number of divisions in i\n{sub_domain_j} number of divisions in j\n{sub_domain_k} number of divisions in k\n"

    # Writing to the .cin file
    try:
        with open('infodom_test.cin', "w") as f:
            
            # Write the header
            f.write(header)
            f.write(equals_line + "\n")
            # Write the DataFrame as a space-separated format with no column or row names, with centered rows
            for index, row in infodom_coord_df.iterrows():
                f.write(f"{row['Domain']}  "
                        f"{row['rdiv']}  "
                        f"{row['x1']}  "
                        f"{row['x2']}  "
                        f"{row['y1']}  "
                        f"{row['y2']}  "
                        f"{row['z1']}  "
                        f"{row['z2']}\n")
            # Add the line of equals signs at the end of the file
            f.write(equals_line + "\n")
            f.write(footer)
    
        # If the file is written successfully, print a success message
        print(f"File '{'infodom_test.cin'}' created successfully!")
    
    except Exception as e:
        # Handle any errors that occur during file creation
        print(f"An error occurred while creating the file: {e}")
        
    return 

In [167]:
create_infodom()

Writing file...
File 'infodom_test.cin' created successfully!


Create the mdmap.cin file.
This will assign processors to each domain.

In [41]:
#ask for user input: How many sub-domains do you want to create?
def create_mdmap_file_with_columns():
    # Collecting user input for the number of domains and processors
    print("Creating mdmap file.")
    while True:
        try:
            num_domains = int(input("How many sub-domains do you want to create? "))
            if num_domains <= 0:
                print("Please enter a number greater than 0.")
            else:
                break  # Exit loop if a valid number is entered
        except ValueError:
            print("Invalid input! Please enter a valid number.")
        
    while True:
        try:
            num_processors = int(input("How many processors do you want assigned to each domain? (The default is 1)"))
            if num_processors <= 0:
                print("Please enter a number greater than 0.")
            else:
                break  # Exit loop if a valid number is entered
        except ValueError:
            print("Invalid input! Please enter a valid number.")

    #create processor_num column for file
    num_processor_list = []
    current_processor_num = 0
    while current_processor_num < num_domains:
        num_processor_list.append(num_processors)
        current_processor_num+=1

    #Update user on status of application
    print('Creating file...')
    #create ID columns
    #make a list of incrementing numbers * number of processors
    total_processors_num = num_domains*num_processors
    domains_processors_ID_list = list(range(total_processors_num))
    #create an entry separated by a semi-colon
    # Split the list and join with semicolon
    ID_list = [
        " ".join(map(str, domains_processors_ID_list[i:i + num_processors]))
        for i in range(0, len(domains_processors_ID_list), num_processors)
    ]
    domain_processor_df = pd.DataFrame({'Sub-Domain ID': ID_list,
        'Num Processors': num_processor_list, 'Processor ID': ID_list})

    #Update user on status of application
    print('Writing file...')
    # Hard-code the length of the equal signs line to 49
    equals_line = "=" * 49

    header = f"{num_domains} number of domains\n{num_processors} number of processors\n{equals_line}\n"

    # Writing to the .cin file
    try:
        with open('mdmap_test.cin', "w") as f:
            # Write the header
            f.write(header)
        
            # Write the DataFrame as a space-separated format with no column or row names, with centered rows
            for index, row in domain_processor_df.iterrows():
                f.write(f"{str(row['Sub-Domain ID']).rjust(4)}  "
                        f"{str(row['Num Processors']).rjust(4)}  "
                        f"{str(row['Processor ID']).rjust(4)}\n")
            # Add the line of equals signs at the end of the file
            f.write(equals_line + "\n")
    
        # If the file is written successfully, print a success message
        print(f"File '{'mdmap_test.cin'}' created successfully!")
    
    except Exception as e:
        # Handle any errors that occur during file creation
        print(f"An error occurred while creating the file: {e}")
        
    return 

In [ ]:
# Call the function to create the .cin file
create_mdmap_file_with_columns()

LPT.cin file is an optional input that is only explored if the user ticks Lagrangian particles in the control.cin file.
Most values are defaults. Fraction is repeated based on user input.

In [ ]:
#general parameters inputs
lagrangian_num = int(input("How many lagrangian fractions do you want?"))
particle_to_particle = input("Do you want particle to particle collisions on or off?")
particle_to_wall = input("Do you want particle to wall collisions on or off?")
#TO DO: ask about stiffness constant - it's not in the LPT file on Github
#general parameters defaults
PSI_cell_ball = "F"
delta_function = 5

In [ ]:
#fractions inputs
release_frequency = int(input("What release frequency do you want?"))
particles_per_release = int(input("How many particles per release do you want?"))
#fractions defaults
particle_diameter = 0.005
std_dev = 0
particle_density = 1320
std_dev_density = 100
release_volume_lx = 0.2
release_volume_ly = 0.2
release_volume_lz = 0.05
spherical_volume_release = "F"
generate_on_surface = "F"
radius = 0.1
spherical_release_options = 1
random_release = "T"
xp = 0.1
yp = 0.125
up = 0.125
vp = 0 
unknown_p = 0
wp = 0.05
#TODO: ask about 6th variable with only 5 keywords

Center files so the format is consistent.

In [42]:
# Function to center the entire file after writing it, takes file_path as argument
def center_file(file_path):
    # Read the content of the file
    with open(file_path, 'r') as f:
        content = f.readlines()

    # Find the max length of any line
    max_width = max(len(line) for line in content)

    #Update user on status of application
    print("Centering content...")
    
    # Center align every line in the file content
    centered_content = [line.strip().center(max_width) + "\n" for line in content]

     # Write the centered content back to the file with try-except for error handling
    try:
        with open(file_path, 'w') as f:
            f.writelines(centered_content)
        print(f"File centered successfully. Your file {file_path} is now in your working directory.")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")
    

In [168]:
center_file('mdmap_test.cin')
center_file('infodom_test.cin')

Centering content...
File centered successfully. Your file mdmap_test.cin is now in your working directory.
Centering content...
File centered successfully. Your file infodom_test.cin is now in your working directory.
